In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb


In [32]:
df = pd.read_csv("fraud_data.csv")

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   client_id                  50000 non-null  object 
 1   account_type               50000 non-null  object 
 2   deposit_amount             50000 non-null  int64  
 3   withdrawal_amount          50000 non-null  int64  
 4   num_trades                 50000 non-null  int64  
 5   avg_trade_amount           50000 non-null  int64  
 6   trade_duration             50000 non-null  int64  
 7   total_profit               50000 non-null  int64  
 8   fees_paid                  50000 non-null  float64
 9   payment_method             50000 non-null  object 
 10  deposit_percentile         50000 non-null  float64
 11  withdrawal_percentile      50000 non-null  float64
 12  num_trades_percentile      50000 non-null  float64
 13  trade_duration_percentile  50000 non-null  flo

In [19]:
df

,client_id,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method,deposit_percentile,withdrawal_percentile,num_trades_percentile,trade_duration_percentile,profit_percentile,risk_score,risk_level,country
0,C00001,VIP,15895,11398,12,306,330,-1611,315.05,E-Wallet,32.017,56.480,25.049,55.486,22.514,54.9890,No Risk,Thailand
1,C00002,Standard,960,502,10,457,108,-2462,15.97,Bank Transfer,1.652,5.423,20.933,17.921,16.864,42.5001,No Risk,Japan
2,C00003,Standard,38258,16044,26,798,214,1122,478.07,Credit Card,76.863,68.536,53.138,36.024,40.923,64.0087,Low Risk,Germany
3,C00004,Standard,44832,21772,1,762,273,6963,753.85,Bank Transfer,89.657,80.016,3.118,46.034,79.815,74.1243,Medium Risk,Japan
4,C00005,Premium,11384,7900,39,546,548,-1632,111.81,Credit Card,22.826,44.876,78.980,91.540,22.376,31.6864,No Risk,Netherlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,C49996,Standard,49049,32351,31,79,323,4885,640.01,Bank Transfer,98.052,92.854,63.068,54.339,66.081,67.3771,Low Risk,Thailand
49996,C49997,Standard,3357,2852,6,440,146,9391,37.92,Credit Card,6.436,21.813,12.923,24.618,95.931,40.7298,No Risk,Australia
49997,C49998,VIP,39329,31974,13,818,501,8123,505.79,Crypto,78.904,92.596,27.108,83.622,87.390,62.6746,Low Risk,Philippines
49998,C49999,VIP,46643,32314,0,862,194,6761,313.55,Credit Card,93.199,92.828,1.055,32.839,78.414,81.8680,High Risk,Pakistan


In [20]:
df_train = df.drop(columns=['client_id', 'risk_score'])
df_train

,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method,deposit_percentile,withdrawal_percentile,num_trades_percentile,trade_duration_percentile,profit_percentile,risk_level,country
0,VIP,15895,11398,12,306,330,-1611,315.05,E-Wallet,32.017,56.480,25.049,55.486,22.514,No Risk,Thailand
1,Standard,960,502,10,457,108,-2462,15.97,Bank Transfer,1.652,5.423,20.933,17.921,16.864,No Risk,Japan
2,Standard,38258,16044,26,798,214,1122,478.07,Credit Card,76.863,68.536,53.138,36.024,40.923,Low Risk,Germany
3,Standard,44832,21772,1,762,273,6963,753.85,Bank Transfer,89.657,80.016,3.118,46.034,79.815,Medium Risk,Japan
4,Premium,11384,7900,39,546,548,-1632,111.81,Credit Card,22.826,44.876,78.980,91.540,22.376,No Risk,Netherlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Standard,49049,32351,31,79,323,4885,640.01,Bank Transfer,98.052,92.854,63.068,54.339,66.081,Low Risk,Thailand
49996,Standard,3357,2852,6,440,146,9391,37.92,Credit Card,6.436,21.813,12.923,24.618,95.931,No Risk,Australia
49997,VIP,39329,31974,13,818,501,8123,505.79,Crypto,78.904,92.596,27.108,83.622,87.390,Low Risk,Philippines
49998,VIP,46643,32314,0,862,194,6761,313.55,Credit Card,93.199,92.828,1.055,32.839,78.414,High Risk,Pakistan


In [21]:
#encoder = LabelEncoder()
#df_train["region"] = encoder.fit_transform(df_train["region"])
#df_train["account_type"] = encoder.fit_transform(df_train["account_type"])
#df_train["payment_method"] = encoder.fit_transform(df_train["payment_method"])

#risk_order = ['No Risk', 'Low Risk', 'Medium Risk', 'High Risk']
#ordinal_encoder = OrdinalEncoder(categories=[risk_order])
#df_train["risk_level"] = ordinal_encoder.fit_transform(df_train[["risk_level"]])


In [22]:
categorical_features = ["country", "account_type", "payment_method"]
# Convert categorical features to category dtype
for col in categorical_features:
    df_train[col] = df_train[col].astype("category") 

In [23]:
# Select features and target
X = df_train[["country", "account_type", "deposit_amount", "withdrawal_amount", "num_trades", 
        "avg_trade_amount", "trade_duration", "total_profit", "fees_paid", "payment_method"]]
y = df_train["risk_level"]

In [24]:
#scaler = StandardScaler()
#X = scaler.fit_transform(X)

In [25]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
# Convert to LightGBM dataset
#train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)

In [12]:
# Define model parameters
params = {
    'objective': 'multiclass',
    'num_class': len(y.unique()),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'max_depth': 10,
    'num_leaves': 31,
    'verbose': -1
}

# Train LightGBM model
model = lgb.LGBMClassifier(**params)

In [13]:
model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.102013


LGBMClassifier(max_depth=10, metric='multi_logloss', num_class=4,
               objective='multiclass', verbose=-1)

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Model prediction
y_pred = model.predict(X_test)

# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Display results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.97
Precision: 0.95
Recall: 0.94
F1 Score: 0.95


In [15]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

# Define model parameters
params = {
    'objective': 'multiclass',
    'num_class': len(y.unique()),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'max_depth': 10,
    'num_leaves': 31,
    'verbose': -1
}

# Initialize LightGBM Classifier
model = lgb.LGBMClassifier(**params)

# Define Stratified k-Fold cross-validation (5 folds)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store scores
train_scores = []
val_scores = []

# Perform k-fold cross-validation
for train_idx, val_idx in kf.split(X, y):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    # Train model on the training fold
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
    )
    
    # Evaluate on training and validation sets
    train_acc = model.score(X_train_fold, y_train_fold)
    val_acc = model.score(X_val_fold, y_val_fold)
    
    train_scores.append(train_acc)
    val_scores.append(val_acc)

# Compute mean and standard deviation of scores
train_mean = np.mean(train_scores)
val_mean = np.mean(val_scores)
train_std = np.std(train_scores)
val_std = np.std(val_scores)

# Display results
print(f"Training Accuracy: {train_mean:.4f} ± {train_std:.4f}")
print(f"Validation Accuracy: {val_mean:.4f} ± {val_std:.4f}")

# Check for overfitting
if train_mean - val_mean > 0.05:
    print("⚠️ Possible Overfitting Detected: Training accuracy is significantly higher than validation accuracy.")
else:
    print("✅ No major overfitting detected: Training and validation scores are close.")



Training Accuracy: 0.9978 ± 0.0001
Validation Accuracy: 0.9709 ± 0.0007
✅ No major overfitting detected: Training and validation scores are close.


In [16]:
import joblib

# Save the trained model
joblib.dump(model, "fraud_model.pkl")

['fraud_model.pkl']

In [ ]:
#X_test.to_csv('X_test.csv', index=False) 

In [26]:
X_test

,country,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method
33553,Russia,Premium,24649,1876,17,479,273,4470,338.16,Bank Transfer
9427,UK,VIP,16238,6407,5,38,125,5991,47.07,Credit Card
199,South Africa,Premium,44517,8837,40,763,141,-1810,403.10,Bank Transfer
12447,Italy,VIP,25495,5938,40,552,454,6599,351.41,Credit Card
39489,Brazil,Standard,11541,6990,20,783,425,8731,126.97,Crypto
...,...,...,...,...,...,...,...,...,...,...
15168,Canada,Premium,7276,5522,0,142,122,616,130.77,E-Wallet
49241,Spain,VIP,36328,1742,15,970,362,751,198.10,E-Wallet
39317,Germany,VIP,41697,19171,16,740,528,-1986,435.29,Bank Transfer
42191,Russia,VIP,14767,4531,22,471,333,5353,26.65,Bank Transfer


In [28]:
X_test.loc[199]

country               South Africa
account_type               Premium
deposit_amount               44517
withdrawal_amount             8837
num_trades                      40
avg_trade_amount               763
trade_duration                 141
total_profit                 -1810
fees_paid                    403.1
payment_method       Bank Transfer
Name: 199, dtype: object

In [ ]:
y_test[y_test == "No Risk"]

In [35]:
# Select features and target
X = df[['client_id', "country", "account_type", "deposit_amount", "withdrawal_amount", "num_trades", 
        "avg_trade_amount", "trade_duration", "total_profit", "fees_paid", "payment_method"]]
y = df["risk_level"]

In [36]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [37]:
df_combined = pd.concat([X_test, y_test], axis=1)



In [96]:
x = df_combined[df_combined['risk_level'].isin(['High Risk'])]
y = df_combined[df_combined['risk_level'].isin(['Low Risk'])]
z = df_combined[df_combined['risk_level'].isin(['Medium Risk'])]
w = df_combined[df_combined['risk_level'].isin(['No Risk'])]

In [102]:
x.iloc[0:10]

,client_id,country,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method,risk_level
21123,C21124,Pakistan,Standard,42405,31130,1,731,219,4129,366.72,Credit Card,High Risk
32207,C32208,Nigeria,VIP,48626,31057,26,58,76,1351,117.39,E-Wallet,High Risk
15079,C15080,Pakistan,Premium,42916,23845,6,721,30,3218,460.80,Bank Transfer,High Risk
30440,C30441,Bangladesh,VIP,39540,23544,16,158,141,-3063,435.37,E-Wallet,High Risk
35557,C35558,Bangladesh,Premium,47212,41632,16,119,213,5384,364.71,Crypto,High Risk
8893,C08894,Vietnam,Standard,46029,24652,11,185,282,-573,105.00,E-Wallet,High Risk
45892,C45893,India,VIP,43882,34932,3,65,20,958,191.83,Bank Transfer,High Risk
38249,C38250,Pakistan,VIP,29311,25692,4,913,174,-4710,334.80,Crypto,High Risk
6943,C06944,India,VIP,42724,27359,9,191,8,715,99.74,Bank Transfer,High Risk
1122,C01123,India,Premium,39811,27718,1,145,264,1094,592.58,Crypto,High Risk


In [104]:
y.iloc[0:10]

,client_id,country,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method,risk_level
199,C00200,South Africa,Premium,44517,8837,40,763,141,-1810,403.10,Bank Transfer,Low Risk
1414,C01415,Italy,Standard,26765,21308,1,960,418,7060,119.48,Bank Transfer,Low Risk
15118,C15119,Australia,Premium,35211,15505,4,85,248,5978,212.69,Bank Transfer,Low Risk
26497,C26498,Brazil,Standard,28055,19917,1,936,435,-28,144.91,Credit Card,Low Risk
30188,C30189,Germany,Standard,39984,21040,45,998,47,-167,95.15,Crypto,Low Risk
38478,C38479,Japan,Premium,47861,18918,30,317,594,-4836,361.90,Crypto,Low Risk
26335,C26336,Netherlands,Premium,17429,12308,14,558,18,4852,272.72,E-Wallet,Low Risk
24712,C24713,Italy,Premium,36631,8724,17,966,193,1084,236.36,Credit Card,Low Risk
40485,C40486,Brazil,Premium,43305,15005,18,751,306,1902,562.79,E-Wallet,Low Risk
17240,C17241,UK,VIP,27627,26284,38,287,119,-1016,146.25,E-Wallet,Low Risk


In [105]:
z.iloc[0:10]

,client_id,country,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method,risk_level
42724,C42725,South Korea,VIP,46257,44563,37,609,236,-4939,686.66,Crypto,Medium Risk
38695,C38696,Philippines,Premium,46458,44225,24,156,421,920,432.40,Credit Card,Medium Risk
47609,C47610,Russia,Premium,45563,36515,20,906,290,822,447.39,Bank Transfer,Medium Risk
15267,C15268,France,Standard,34990,12960,10,303,35,8417,513.38,Bank Transfer,Medium Risk
30001,C30002,Russia,Premium,36315,31394,1,24,495,1468,692.89,Bank Transfer,Medium Risk
4892,C04893,Italy,Standard,47617,12705,25,26,146,-1060,572.71,Credit Card,Medium Risk
22949,C22950,Spain,Premium,46408,15395,22,700,207,-3604,569.84,Crypto,Medium Risk
20304,C20305,Thailand,VIP,27090,23812,31,681,50,-3023,67.73,E-Wallet,Medium Risk
8270,C08271,Russia,VIP,46389,40103,18,636,514,-3745,354.24,Crypto,Medium Risk
13624,C13625,Malaysia,Premium,46568,8591,13,357,172,-3121,601.46,Crypto,Medium Risk


In [106]:
w.iloc[0:10]

,client_id,country,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method,risk_level
33553,C33554,Russia,Premium,24649,1876,17,479,273,4470,338.16,Bank Transfer,No Risk
9427,C09428,UK,VIP,16238,6407,5,38,125,5991,47.07,Credit Card,No Risk
12447,C12448,Italy,VIP,25495,5938,40,552,454,6599,351.41,Credit Card,No Risk
39489,C39490,Brazil,Standard,11541,6990,20,783,425,8731,126.97,Crypto,No Risk
10822,C10823,Thailand,VIP,752,608,38,637,381,5969,14.95,Bank Transfer,No Risk
49498,C49499,Mexico,VIP,31558,9716,46,943,261,9592,79.01,Crypto,No Risk
4144,C04145,Germany,Standard,27333,18181,22,963,386,-1021,396.20,Bank Transfer,No Risk
36958,C36959,Australia,Standard,2018,1951,42,526,2,3636,29.24,Credit Card,No Risk
43106,C43107,Mexico,VIP,26093,10057,15,268,293,8931,395.65,Bank Transfer,No Risk
6188,C06189,Spain,Premium,38851,5494,30,165,596,4080,640.38,Crypto,No Risk


In [47]:
df_combined.to_csv('test.csv', index=False) 

In [86]:
df_combined.head(50)

,client_id,country,account_type,deposit_amount,withdrawal_amount,num_trades,avg_trade_amount,trade_duration,total_profit,fees_paid,payment_method,risk_level
33553,C33554,Russia,Premium,24649,1876,17,479,273,4470,338.16,Bank Transfer,No Risk
9427,C09428,UK,VIP,16238,6407,5,38,125,5991,47.07,Credit Card,No Risk
199,C00200,South Africa,Premium,44517,8837,40,763,141,-1810,403.10,Bank Transfer,Low Risk
12447,C12448,Italy,VIP,25495,5938,40,552,454,6599,351.41,Credit Card,No Risk
39489,C39490,Brazil,Standard,11541,6990,20,783,425,8731,126.97,Crypto,No Risk
42724,C42725,South Korea,VIP,46257,44563,37,609,236,-4939,686.66,Crypto,Medium Risk
10822,C10823,Thailand,VIP,752,608,38,637,381,5969,14.95,Bank Transfer,No Risk
49498,C49499,Mexico,VIP,31558,9716,46,943,261,9592,79.01,Crypto,No Risk
4144,C04145,Germany,Standard,27333,18181,22,963,386,-1021,396.20,Bank Transfer,No Risk
36958,C36959,Australia,Standard,2018,1951,42,526,2,3636,29.24,Credit Card,No Risk
